# Capstone 2: Personality Prediction MBIT

## Proposal
Have you ever been curious about the personality type of yourself and people around 
you? Is there any simple test to help us find the answer without seeing a psychologist? The answer is yes. You can figure out your personality type by simply answering these four questions:

* Are you outwardly or inwardly focused ? 
* How do you prefer to take in information? 
* How do you prefer to make decisions? 
* How do you prefer to live your outer life?

For each question, there are two options to choose from. You choose the side that seems most  natural for you. The options are: Introversion or Extraversion, Sensing or Intuition, Thinking or Feeling and Judging or Perception respectively. The introspective self-report questionnaire we introduced above is known as Myers-Briggs -Type- Indicator (MBTI). It is a personality type system that divides everyone into 16 distinct personalities based one their answers to the previous questions. For example, someone who chooses extraversion, sensing, thinking and judging would be labeled an ESTP in the MBTI system. 

This project focuses on using machine learning algorithms to predict the personality of a person from the type of posts they put on social media. The data was collected through the PersonalityCare forum and is available on Kaggle Personality Prediction Dataset. This dataset consists of over 8600 rows representing 8600 different people and 2 columns representing a person’s MBTI personality type and the things they have posted.
